In [ ]:
import pandas as pd
import mlflow
import os

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
# leer el dataframe y establecer los nombres de las columnas
DATA_DIR = "datos/covertype.csv"
column_names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type', 'Cover_Type']
data = pd.read_csv(DATA_DIR, names=column_names, header=None)

# eliminar duplicados en los datos
data.drop_duplicates(inplace=True, ignore_index=True)

# mostrar una muestra de los datos
data.head()


In [ ]:
# almacenar la variable objetivo como un vector y
target_variable = "Cover_Type"
y = data[target_variable]

# establecer las variables de entrada X como los datos originales sin la variable objetivo ni las variables categóricas
X = data.drop(columns=[target_variable])

# transformar las variables para mejorar el rendimiento del modelo
categorical_variables = ['Wilderness_Area', 'Soil_Type']
numerical_variables = [feature for feature in X.columns if feature not in categorical_variables]

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OrdinalEncoder(), categorical_variables),
        ('numerical', StandardScaler(), numerical_variables)
    ]
)

In [ ]:
# definir el clasificador como una máquina de gradiente boosting y combinarlo con el preprocesador
classifier = HistGradientBoostingClassifier(categorical_features=[0, 1])

model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', classifier)
])

In [ ]:
# configurar variables de entorno para MinIO
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.101.108:8081"
os.environ['AWS_ACCESS_KEY_ID'] = 'minio_user'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio_pwd'

# conectar con MLflow
mlflow.set_tracking_uri("http://10.43.101.108:8083")
mlflow.set_experiment("gradient_boosting_covertype")
mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True, registered_model_name="final_model")

# ejecutar MLflow
with mlflow.start_run(run_name="gradient_boosting_training", description="Entrenar modelo de gradient boosting") as run:
    # ajustar el modelo a los datos
    model.fit(X, y)